# Multi-Agent Creative Systems

## Setup

In [ ]:
# Install required packages
!pip install -q langchain langchain-core torch
!pip install -q pydantic pronouncing nltk
!pip install -U "langchain[google-genai]"

In [ ]:
# Core imports
import os
from typing import TypedDict, List, Optional, Any, Dict, Literal
from pydantic import BaseModel, Field

# LangChain imports
from langchain.chat_models import init_chat_model
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.messages import HumanMessage, SystemMessage, AIMessage
from langchain.agents import create_agent
from langchain.tools import tool
from langchain.agents import (
    create_agent,
    AgentState,
)
from langchain.agents.middleware import (
    AgentMiddleware,
    ModelRequest,
    ModelResponse,
    ToolCallRequest,
    dynamic_prompt,
    wrap_model_call,
    wrap_tool_call,
    before_model,
    after_model
)
from langchain.agents.structured_output import ToolStrategy, ProviderStrategy

# Utilities
import pronouncing
import warnings
warnings.filterwarnings('ignore')

print("All libraries imported successfully")

## Initialize Gemini API model

In [ ]:
from getpass import getpass

# Set environment variables
os.environ['GOOGLE_API_KEY'] = getpass('Enter your Google API key: ')

In [ ]:
base_llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")

print("Model endpoint ready")

## Define tool

In [ ]:
@tool
def find_rhymes(word: str) -> str:
    """Find rhyming words for creative writing. Returns comma-separated list of rhymes.

    Args:
        word: The word to find rhymes for

    Returns:
        A comma-separated list of rhyming words
    """
    word = word.strip().lower()
    rhymes = pronouncing.rhymes(word)

    if not rhymes:
        return f"No rhymes found for '{word}'"

    # Return top 10 rhymes
    top_rhymes = rhymes[:10]
    return ", ".join(top_rhymes)


@tool
def count_syllables(text: str) -> str:
    """Count syllables in a word or phrase for poetry and rhythm analysis.

    Args:
        text: The word or phrase to analyze

    Returns:
        Total syllable count and breakdown by word
    """
    text = text.strip().lower()
    words = text.split()

    total_syllables = 0
    word_counts = []

    for word in words:
        # Remove punctuation
        word = ''.join(c for c in word if c.isalnum())
        phones = pronouncing.phones_for_word(word)

        if phones:
            syllable_count = pronouncing.syllable_count(phones[0])
            total_syllables += syllable_count
            word_counts.append(f"{word}({syllable_count})")
        else:
            # Estimate for unknown words
            estimate = max(1, len([c for c in word if c in 'aeiou']))
            total_syllables += estimate
            word_counts.append(f"{word}(~{estimate})")

    return f"Total: {total_syllables} syllables. Breakdown: {' '.join(word_counts)}"


@tool
def analyze_meter(line: str) -> str:
    """Analyze the poetic meter of a line of text.

    Args:
        line: A line of poetry or dialogue to analyze

    Returns:
        Stress pattern and meter type
    """
    words = line.strip().lower().split()
    stresses = []

    for word in words:
        word = ''.join(c for c in word if c.isalnum())
        phones = pronouncing.phones_for_word(word)

        if phones:
            stress = pronouncing.stresses(phones[0])
            stresses.append(stress)

    pattern = ''.join(stresses)

    # Simple meter detection
    if pattern.count('10') > 2:
        meter = "iambic (weak-strong)"
    elif pattern.count('01') > 2:
        meter = "trochaic (strong-weak)"
    else:
        meter = "mixed or free verse"

    return f"Pattern: {pattern}. Meter: {meter}"


# List of all creative tools
creative_tools = [find_rhymes, count_syllables, analyze_meter]

print("✓ Tools defined:")
for tool_obj in creative_tools:
    print(f"  - {tool_obj.name}: {tool_obj.description[:60]}...")

In [ ]:
from langchain.messages import ToolMessage

@wrap_tool_call
def handle_tool_errors(request, handler):
    """Handle tool execution errors with custom messages."""
    try:
        return handler(request)
    except Exception as e:
        # Return a custom error message to the model
        return ToolMessage(
            content=f"Tool error: Please check your input and try again. ({str(e)})",
            tool_call_id=request.tool_call["id"]
        )

In [ ]:
playwright_agent = create_agent(
    model=base_llm,
    tools=creative_tools,
    system_prompt="""You are a poet and writer. Don't explain or ask, just produce the asked result.
    When creating text that requires that words rhyme, use the tools to find rhyming words.
    Also use the tool to insure rythmic validity of the poem.

Use the provided tools to:
- Find rhymes for poetic dialogue
- Count syllables for rhythm
- Analyze meter for dramatic effect
""",
    middleware=[handle_tool_errors]
)

In [ ]:
prompt = "Write me a poem about AI discovering love, use the tools at your disposition"

result = playwright_agent.invoke(
    {"messages": [{"role": "user", "content": prompt}]}
)

script_text = result["messages"][-1].content

print(result)
print(script_text)

## Multi-agent system

In [ ]:
# idea generation
idea_agent = create_agent(
    model=base_llm,
    tools=[],
    system_prompt=SystemMessage(
        content=(
            "You are a creative ideation assistant.\n"
            "Generate a strong, original concept for a creative text.\n"
            "Output only the idea, no prose."
        )
    ),
)

# writing
writer_agent = create_agent(
    model=base_llm,
    tools=[],
    system_prompt=SystemMessage(
        content=(
            "You are a creative writer.\n"
            "Write a complete creative text based on the provided idea.\n"
            "Output only the creative content."
        )
    ),
)

# editing
editor_agent = create_agent(
    model=base_llm,
    tools=[],
    system_prompt=SystemMessage(
        content=(
            "You are a literary editor.\n"
            "Improve style, flow, and emotional impact.\n"
            "Do not add commentary.\n"
            "Output only the revised creative text."
        )
    ),
)


In [ ]:

# orchestrating everything

user_prompt = "A short piece of speculative fiction about memory loss."

# Step 1: generate idea
idea_result = idea_agent.invoke({
    "messages": [HumanMessage(content=user_prompt)]
})
idea = idea_result["messages"][-1].content

# Step 2: write draft
draft_result = writer_agent.invoke({
    "messages": [HumanMessage(content=idea)]
})
draft = draft_result["messages"][-1].content

# Step 3: edit draft
final_result = editor_agent.invoke({
    "messages": [HumanMessage(content=draft)]
})

final_text = final_result["messages"][-1].content

In [ ]:
final_text = final_result["messages"][-1].content

In [ ]:
def print_block(title: str, content: str):
    line = "═" * 80
    print(f"\n{line}")
    print(f"{title}")
    print(line)
    print(content.strip())
    print(line)

In [ ]:
print_block(
    "USER PROMPT",
    user_prompt
)

print_block(
    "AGENT 1 — IDEA GENERATOR OUTPUT",
    idea
)

print_block(
    "AGENT 2 — WRITER DRAFT",
    draft
)

print_block(
    "AGENT 3 — EDITED FINAL TEXT",
    final_text
)

## Improvements

### User context

In [ ]:
from typing import TypedDict
from langchain.agents import create_agent
from langchain.agents.middleware import dynamic_prompt, ModelRequest
from langchain.messages import HumanMessage


# -------------------------
# Context schema
# -------------------------
class Context(TypedDict):
    user_role: str
    user_name: str
    include_user_in_story: bool


# -------------------------
# Dynamic system prompt
# -------------------------
@dynamic_prompt
def user_role_prompt(request: ModelRequest) -> str:
    role = request.runtime.context.get("user_role", "user")
    name = request.runtime.context.get("user_name", "the user")
    include = request.runtime.context.get("include_user_in_story", False)

    base = "You are a creative writing assistant."

    if include:
        base += f" You may include {name} as a character if appropriate."

    if role == "expert":
        return base + " Use rich language and experimental structure."
    elif role == "beginner":
        return base + " Keep the language simple and accessible."

    return base


# -------------------------
# Agent
# -------------------------
creative_agent = create_agent(
    model=base_llm,
    tools=[],
    middleware=[user_role_prompt],
    context_schema=Context,
)


In [ ]:
def print_block(title, content):
    line = "═" * 80
    print(f"\n{line}")
    print(title)
    print(line)
    print(content.strip())
    print(line)


user_prompt = "Write a short speculative story about forgotten memories."

print_block("🧠 USER PROMPT", user_prompt)

result = creative_agent.invoke(
    {"messages": [HumanMessage(content=user_prompt)]},
    context={
        "user_role": "expert",
        "user_name": "Paul",
        "include_user_in_story": True,
    }
)

final_text = result["messages"][-1].content
print_block("✍️ SINGLE AGENT OUTPUT", final_text)


### Human in the loop

In [ ]:
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver
from langchain.messages import HumanMessage


@tool
def validate(idea: str):
  """
  Validate the idea with the user
  """
  return

idea_agent = create_agent(
    model=base_llm,
    tools=[validate],
    system_prompt=(
        "You are an idea generator.\n"
        "Propose ONE strong creative idea in 2–3 sentences.\n"
        "No prose, no writing yet.\n"
        "You must get human approval before you move forward with the idea.\n"
        "Do this by invoking the validate tool"
    ),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "validate": True
            }
        )
    ],
    checkpointer=InMemorySaver(),
)

In [ ]:
print_block("💡 IDEA GENERATION", "Generating idea, I want to first.")

config = {"configurable": {"thread_id": "thread-1"}}

idea_result = idea_agent.invoke({
    "messages": [HumanMessage(content="Generate a speculative fiction idea.")]
},config=config)

print(idea_result)

idea = idea_result["messages"][-1].content
print_block("💡 APPROVED IDEA", idea)

### Multi-agent (graph)

In [ ]:
from langchain.tools import tool


idea_subagent = create_agent(
    model=base_llm,
    tools=[],
    system_prompt="Generate a strong creative story idea."
)

writer_subagent = create_agent(
    model=base_llm,
    tools=[],
    system_prompt="Write a complete creative text from the given idea."
)

editor_subagent = create_agent(
    model=base_llm,
    tools=[],
    system_prompt="Edit the text for clarity, flow, and emotional impact."
)

In [ ]:
@tool("generate_idea", description="Generate a creative story idea")
def generate_idea(prompt: str) -> str:
    result = idea_subagent.invoke({
        "messages": [{"role": "user", "content": prompt}]
    })
    return result["messages"][-1].content


@tool("write_story", description="Write a story from an idea")
def write_story(idea: str) -> str:
    result = writer_subagent.invoke({
        "messages": [{"role": "user", "content": idea}]
    })
    return result["messages"][-1].content


@tool("edit_story", description="Edit and polish a story")
def edit_story(text: str) -> str:
    result = editor_subagent.invoke({
        "messages": [{"role": "user", "content": text}]
    })
    return result["messages"][-1].content


In [ ]:
supervisor = create_agent(
    model=base_llm,
    tools=[generate_idea, write_story, edit_story],
    system_prompt=(
        "You are a supervisor coordinating creative subagents.\n"
        "Workflow:\n"
        "1. Generate idea\n"
        "2. Write story\n"
        "3. Edit story\n"
        "Return only the final edited text\n"
        " after generating the idea, then writing the story, and editing it"
    ),
)


In [ ]:
print_block("🧠 USER PROMPT", "Write a short speculative story about memory loss.")

result = supervisor.invoke({
    "messages": [{
        "role": "user",
        "content": "Write a short speculative story about memory loss."
    }]
})

print(result)
final_story = result["messages"][-1].content
print_block("📖 FINAL STORY", final_story)


In [ ]:

def pretty_print_messages(messages):
    print("="*80)
    print("💬 Conversation Trace")
    print("="*80)

    for idx, msg in enumerate(messages, start=1):
        if isinstance(msg, HumanMessage):
            role = "USER"
        elif isinstance(msg, AIMessage):
            role = "AI"
        elif isinstance(msg, ToolMessage):
            role = f"TOOL ({msg.name})" if hasattr(msg, "name") else "TOOL"
        else:
            role = "UNKNOWN"

        print(f"\n[{idx}] {role}")
        print("-"*80)
        # Content
        content = getattr(msg, "content", "")
        print(content.strip() if content else "<No content>")

        # Additional info
        if hasattr(msg, "tool_call_id") and msg.tool_call_id:
            print(f"\nTool call id: {msg.tool_call_id}")

        if hasattr(msg, "tool_calls") and msg.tool_calls:
            print(f"\nTool calls:")
            for call in msg.tool_calls:
                print(f"   - {call['name']}({call['args']}) -> id={call['id']}")

        if hasattr(msg, "response_metadata") and msg.response_metadata:
            model = msg.response_metadata.get("model_name")
            reason = msg.response_metadata.get("finish_reason")
            if model or reason:
                print(f"\nModel metadata: {model}, finish_reason: {reason}")
        print("-"*80)

In [ ]:
pretty_print_messages(result["messages"])